In [1]:
import pandas as pd
import csv
import sys
import re
import scipy
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from time import time

csv.field_size_limit(sys.maxsize)

131072

In [2]:
metrics = ['cbo','wmc','dit','noc','rfc','lcom','nom','nopm','nosm','nof','nopf','nosf','nosi','loc']

def get_metrics(row):
    features = []
    for metric in metrics:
        features.append(row[metric])
    return features

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

param_dist = {'C': scipy.stats.expon(scale=100)}

lr_classifier = LogisticRegression(random_state=42)

n_iter_search = 20
random_search = RandomizedSearchCV(lr_classifier,
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   cv=5,
                                   n_jobs=-1)

In [3]:
df = pd.read_pickle('../data/instances.pkl')
labels = list(set(df['target'].values))

In [4]:
X_train = []
Y_train = []
X_test = []
Y_test = []

print("Preparing lists...")
for index, row in df.iterrows():
    X_train.append(get_metrics(row))
    Y_train.append(row["target"])

X_t, X_v, y_t, y_v = train_test_split(train_instances, Y_train, train_size = 0.75)

start = time()
print("Hyperparameter tuning...")
random_search.fit(train_instances, Y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
  " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)
# print("============ EVALUATION on test set:")
# print(accuracy_score(Y_test, random_search.best_estimator_.predict(test_instances)))

Preparing lists...
[['14', '4', '1', '0', '9', '0', '3', '2', '0', '2', '0', '0', '0', '91']]


NameError: name 'train_instances' is not defined